Mike Fiddler Core 8/21/22

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [3]:
# set our API call parameters and filename before the first call
LOCATION = 'Boston, MA,02119'
TERM = 'Steak and Cheese'

In [4]:
JSON_FILE_boston = f"Data/results_in_progress_steak_and_cheese.json"
JSON_FILE_boston

'Data/results_in_progress_steak_and_cheese.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE_boston)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE_boston)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE_boston} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE_boston,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE_boston} already exists.")

[i] Data/results_in_progress_steak_and_cheese.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE_boston,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 354 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

354

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

0

In [13]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

ZeroDivisionError: division by zero

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE_boston,'w') as f:
     json.dump(previous_results,f)

In [12]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE_boston, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE_boston,'w') as f:
        json.dump(previous_results,f)

NameError: name 'n_pages' is not defined

In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE_boston)
display(final_df.head(), final_df.tail())

In [ ]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

In [ ]:
final_df.drop_duplicates(subset = 'id', inplace= True)

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_steak_and_cheese.csv.gz', compression='gzip',index=False)